In [10]:
import numpy as np
import pandas as pd
import random

def age_series(age = 30, periods_per_year = 12, years = 40):
    age_array = np.ones(periods_per_year * years + 1) / periods_per_year
    age_array[0] = age
    age_array = np.cumsum(age_array)
    age_series = pd.Series(age_array, name='age')
    return age_series

def salary_series(salary = 100000, salary_growth = .04, periods_per_year = 12, years = 40, age = 30, ret_age = 65):
    income_period = ret_age - age
    salary_growth_per_period = (1+salary_growth)**(1/periods_per_year)-1
    salary_growth_array = np.ones(periods_per_year * years + 1) * (1+salary_growth_per_period)
    salary_growth_array[0] = salary
    salary_array = np.cumprod(salary_growth_array)
    salary_array[(income_period*periods_per_year):] = 0
    salary_series = pd.Series(salary_array, name='salary')
    return salary_series

def invest_series(salary_s = None, invest_pct = .15, periods_per_year = 12, withdraw_amount = 100000, age = 30, ret_age = 65):
    income_period = ret_age - age
    if salary_s is None:
        salary_s = salary_series()
    invest_pct_per_period = invest_pct / periods_per_year
    invest = salary_s * invest_pct_per_period
    invest[income_period*periods_per_year:] = -1 * withdraw_amount / periods_per_year
    invest.name = 'invest'
    return invest

def mortality_sim(age = 32, ret_age = 65, sex = 'F', smoker = 'NS', n_scenarios = 1000, mi = .99):
    
    mort_df = pd.read_csv('MortalityTables/cso2017.csv')
    mort_df = mort_df.set_index(['Sex', 'SmokingStatus', 'IssueAge'])
    row = mort_df.loc[(sex, smoker, ret_age)]
    arr = np.empty(n_scenarios)

    for n in range(n_scenarios):
        d = 0
        j = ret_age
        i = row[d]/1000 * (mi ** (j - age))
        while random.random() > i:
            j += 1
            d +=1
            if d < len(row):  # Check if index is out of range
                i = row[d]/1000 * (mi ** (j - age))
            else:
                break
        arr[n] = j

    df = pd.DataFrame(arr, columns=['age_at_death'])

    return df

print(invest_series())

0      1250.000000
1      1254.092175
2      1258.197746
3      1262.316758
4      1266.449255
          ...     
476   -8333.333333
477   -8333.333333
478   -8333.333333
479   -8333.333333
480   -8333.333333
Name: invest, Length: 481, dtype: float64


In [12]:
from allocation import allocation

allocation_df = allocation()
age_s = age_series()
invest_s = invest_series()
salary_s = salary_series()

df = pd.concat([age_s, allocation_df, invest_s, salary_s], axis=1)
print(df)


           age  equity  bond       invest         salary
0    30.000000     0.9   0.1  1250.000000  100000.000000
1    30.083333     0.9   0.1  1254.092175  100327.373978
2    30.166667     0.9   0.1  1258.197746  100655.819694
3    30.250000     0.9   0.1  1262.316758  100985.340655
4    30.333333     0.9   0.1  1266.449255  101315.940382
..         ...     ...   ...          ...            ...
476  69.666667     0.4   0.6 -8333.333333       0.000000
477  69.750000     0.4   0.6 -8333.333333       0.000000
478  69.833333     0.4   0.6 -8333.333333       0.000000
479  69.916667     0.4   0.6 -8333.333333       0.000000
480  70.000000     0.4   0.6 -8333.333333       0.000000

[481 rows x 5 columns]
